If you are attempting an early update and not all metadata is yet available on DBLP - then try to do a manual update via this step

In [1]:
import pandas as pd

last_year_in_dataset = 2022
#last run on 2018 with VAST conferences

#This has to be run once on the data that Natkamon cleaned including all authors up to 2016
#deduped_vispubdata_entries = pd.read_csv("Deduped-Authors.csv")
#gender_files =  pd.read_csv("authorPaperGender-cleaned.v2-2018-11.csv")
#gender_files.drop('Paper.DOI', axis=1, inplace=True,errors='ignore')
#gender_files.drop('Author.Position.Per.Paper', axis=1, inplace=True,errors='ignore')
#gender_files.drop('PositionRegion', axis=1, inplace=True,errors='ignore')
#gender_files.drop('certainty', axis=1, inplace=True,errors='ignore')

#print(gender_files.info())

#gender_files = gender_files.drop_duplicates()
#print(gender_files.info())

#gender_files.to_csv('gender-authors-cleaned-2016.csv',index=False)

In [2]:
#get the last version of the gender file
last_cleaned_version = pd.read_csv("gender-authors-cleaned-2018-with-VAST-conf.csv")
print(last_cleaned_version.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5690 entries, 0 to 5689
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   author  5690 non-null   object
 1   gender  5690 non-null   object
dtypes: object(2)
memory usage: 89.0+ KB
None


In [3]:

#Make sure you have made a new version of this file before in Step 3
deduped_authors = pd.read_csv("Deduped-Authors.csv")
print(deduped_authors.info())

last_cleaned_genders = last_cleaned_version['author'].tolist()
deduped_author_list = deduped_authors['Authors'].tolist()

new_authors = []

print("Authors not found in the gender list:")
for a in deduped_author_list:
    if not a in last_cleaned_genders:
        print(a)
        new_authors.append(a)

#now check which authors are NOT in the gender file
#reasons they might not be in there are:
# -- they are new authors
# -- their deduped name has changes




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6104 entries, 0 to 6103
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Authors  6103 non-null   object
dtypes: object(1)
memory usage: 47.8+ KB
None
Authors not found in the gender list:
Rebecca Nowak
Alexandra Voit
Doris Kosminsky
Danqing Shi
Fraser Anderson
Ashley D. Spear
Jian Zhang 0070
David Hysom
Bowen Yu 0004
Gábor Szücs
Nan Chen
Daniel Plakinger
Will Epperson
Seungyong Lee 0001
Andreas Walch
Madison A. Elliott
Dirk Streeb
Audace Nakeshimana
Adam Kunen
Lawrence Lee
Hyeon Jeon
Paula Kayongo
Vanessa Peña Araya
Mingliang Xue
Andrew Cunningham
Chenhui Li
Kostiantyn Kucher
MingJie Tang
Tingkai Sha
Ching-Shan Chou
Laurent Lessard
Wei Zeng 0002
Aditya G. Parameswaran
Markus Wallinger
Jochen Fröhlich
Yitao Wu
William E. Lorenson
Zhutian Chen
Tyler Estro
Dylan Cashman
Yang Yue
Sanja Scepanovic
Lynn McVey
Raymond Huang
Dong Sun 0001
Zezheng Feng
Antonios Somarakis
J. Xavier Pr

Now attempt to assign a gender from the master gender list

In [4]:
file_encoding = 'utf8'        # set file_encoding to the file encoding (utf8, latin1, etc.)
input_fd = open("masterGenderFile.csv", encoding=file_encoding, errors = 'ignore')
master_gender_df = pd.read_csv(input_fd)

print(master_gender_df.info())

new_authors_gender = []
new_authors_certainty = []

for a in new_authors:
    #extract the first name
    firstname = a.split()[0].lower()
    
    #check if the firstname is in the mastergenderdf
    found = master_gender_df['name'].str.contains(firstname).any()
    
    if(found != False):
        rowindex = master_gender_df.index[master_gender_df['name'].str.contains(firstname) == True].tolist()[0]
        gender = master_gender_df.at[rowindex,'gender']
        new_authors_certainty = master_gender_df.at[rowindex,'confidence']
        new_authors_gender.append(gender)
    else:
        new_authors_gender.append("unknown")
        
pd.DataFrame({'author':new_authors,'gender':new_authors_gender}).to_csv('NewAuthorsGender.csv',index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121044 entries, 0 to 121043
Data columns (total 3 columns):
name          121038 non-null object
gender        121044 non-null object
confidence    121043 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.8+ MB
None


The next step is to clean the NewAuthorsGender.csv by hand - and in particular check if some of the unisex and unknown can't be resolved'

In [7]:
new_authors = pd.read_csv("NewAuthorsGender - Cleaned.csv")
last_cleaned_version.rename(index=str, inplace=True, columns={"Author.Names":"author"})

frames = [new_authors, last_cleaned_version]
full_author_gender_file = pd.concat(frames,ignore_index=True)

print(last_cleaned_version.info())
print(new_authors.info())

print(full_author_gender_file.info())

full_author_gender_file.to_csv('gender-authors-cleaned-'+str(last_year_in_dataset)+".csv",index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 5600 entries, 0 to 5599
Data columns (total 2 columns):
author    5600 non-null object
gender    5600 non-null object
dtypes: object(2)
memory usage: 131.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
author    90 non-null object
gender    90 non-null object
dtypes: object(2)
memory usage: 1.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5690 entries, 0 to 5689
Data columns (total 2 columns):
author    5690 non-null object
gender    5690 non-null object
dtypes: object(2)
memory usage: 89.0+ KB
None


Next we need to create the new data file with the author positions and the DOIs for the gender file and then we're done. Wohoo.

In [8]:
vispubdata = pd.read_csv("IEEE VIS papers 1990-2018 - Main dataset.csv")
dois = []
names = []
genders = []
positions = []
position_range = []

for doi in vispubdata['DOI']:
    
    rowindex = vispubdata.index[vispubdata['DOI'] == doi].tolist()[0]
    authors = vispubdata.at[rowindex,'AuthorNames-Deduped']
    #print(authors)
    author_list = authors.split(";")
    index = 1
    pos_range = 'F'
    for a in author_list:
        dois.append(doi)
        names.append(a)
        
        ridx = full_author_gender_file.index[full_author_gender_file['author'] == a].tolist()[0]
        gender = full_author_gender_file.at[ridx,'gender']
        genders.append(gender)
        positions.append(index)
        
        if index == 1:
            position_range.append('F')
        elif index == len(author_list):
            position_range.append('L')
        else:
            position_range.append('M')
        
        index = index + 1

#print(str(len(dois)) + " " + str(len(names)) + " " + str(len(genders)) + " " + str(len(positions)) +" " + str(len(position_range)))
pd.DataFrame({'doi':dois,'author':names,'gender':genders,'position':positions,'position.rank':position_range}).to_csv("authorPaperGender-"+str(last_year_in_dataset)+".csv",index=False)

newdf = pd.DataFrame({'doi':dois,'author':names,'gender':genders,'position':positions,'position.rank':position_range})


In [9]:
newdf[newdf['gender'] == 'unisex'].head(100)

,doi,author,gender,position,position.rank
2402,10.1109/TVCG.2010.221,Xiangmin Zhou,unisex,2,M
2403,10.1109/TVCG.2010.221,Yunhe Shen,unisex,3,M
2955,10.1109/TVCG.2012.239,Tracy Compton,unisex,2,M
3237,10.1109/TVCG.2013.135,Xin Zhao,unisex,1,F
4021,10.1109/TVCG.2014.2346898,Yahui Zhao,unisex,4,M
6078,10.1109/VAST.2006.261425,Yuli Gao,unisex,4,M
6510,10.1109/VAST.2008.4677389,Bai Wang 0001,unisex,6,L
7556,10.1109/VAST.2015.7347675,Haoling Dong,unisex,1,F
7956,10.1109/VISUAL.1991.175784,Ola Engelsen,unisex,4,L
8402,10.1109/VISUAL.1994.346289,Lee S. Elson,unisex,3,M


In [10]:
newdf[newdf['gender'] == 'unknown'].head(100)

,doi,author,gender,position,position.rank
1077,10.1109/TVCG.2006.176,Yinggang Li,unknown,1,F
1204,10.1109/TVCG.2007.70530,Yinggang Li,unknown,3,M
3577,10.1109/TVCG.2013.229,Hee-Joon Bae,unknown,3,M
4377,10.1109/TVCG.2015.2467691,Sixiao Yang,unknown,6,M
4539,10.1109/TVCG.2015.2468151,Sixiao Yang,unknown,4,M
5110,10.1109/TVCG.2017.2744078,Junyoung Choi,unknown,2,M
5111,10.1109/TVCG.2017.2744078,Haejin Jeong,unknown,3,M
5127,10.1109/TVCG.2017.2744098,Fenjin Ye,unknown,2,M
5165,10.1109/TVCG.2017.2744218,Kejian Zhao,unknown,5,M
5228,10.1109/TVCG.2017.2744378,Jiannan Xiao,unknown,2,M
